In [ ]:
import sys
!{sys.executable} -m pip install pandas pybaseball

In [1]:
from pybaseball import playerid_lookup, team_batting, team_pitching, pitching_stats, playerid_lookup, statcast_batter, schedule_and_record

In [19]:
# team xFIP (last 3 seasons)
team_xfip_df = team_pitching(2021, 2023, ind=0)
team_xfip = team_xfip_df['xFIP'].loc[team_xfip_df['Team'] == 'TEX']
team_xfip

20    4.34
Name: xFIP, dtype: float64

In [28]:
# starting pitcher xFIP (last 3 seasons)
starter_xfip_df = pitching_stats(2021, 2023, ind=0)
starter_xfip = starter_xfip_df['xFIP'].loc[starter_xfip_df['Name'] == 'Nathan Eovaldi']
starter_xfip

43    3.63
Name: xFIP, dtype: float64

In [ ]:
# lineup xwOBA (last 3 seasons)
player_id = playerid_lookup('Semien', 'Marcus')['key_mlbam']
lineup_xwoba_df = statcast_batter('2021-01-01', '2024-12-31', player_id=543760)

In [2]:
# training data - game results from 2017-2019

train_data = {
    'team1': [],
    'team1_win': [],
    'team1_team_xwoba': [],
    'team1_lineup_xwoba': [],
    'team1_team_xfip': [],
    'team1_starter_xfip': [],
    'team1_close_win_pct': [],
    'team1_run_diff': [],
    'team2': [],
    'team2_win': [],
    'team2_team_xwoba': [],
    'team2_lineup_xwoba': [],
    'team2_team_xfip': [],
    'team2_starter_xfip': [],
    'team2_close_win_pct': [],
    'team2_run_diff': []
}

In [5]:
def process_games(row):
    train_data['team1'].append(row['Tm'])
    train_data['team1_win'].append(1 if row['W/L'][0] == 'W' else 0)
    # train_data['team1_team_xwoba'].append()
    # train_data['team1_lineup_xwoba'].append()
    # train_data['team1_team_xfip'].append()
    # train_data['team1_starter_xfip'].append()
    # train_data['team1_close_win_pct'].append()
    # train_data['team1_run_diff'].append()
    
    train_data['team2'].append(row['Opp'])
    train_data['team2_win'].append(1 if row['W/L'][0] == 'L' else 0)
    # train_data['team2_team_xwoba'].append()
    # train_data['team2_lineup_xwoba'].append()
    # train_data['team2_team_xfip'].append()
    # train_data['team2_starter_xfip'].append()
    # train_data['team2_close_win_pct'].append()
    # train_data['team2_run_diff'].append()

In [8]:
# arizona diamondbacks
ari_games = schedule_and_record(2017, 'ARI')
ari_games.apply(process_games, axis=1)
ari_games

http://www.baseball-reference.com/teams/ARI/2017-schedule-scores.shtml


/home/jeremy/repo/game-prediction-models/env/lib/python3.12/site-packages/pybaseball/team_results.py:75: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Attendance'].replace(r'^Unknown$', np.nan, regex=True, inplace = True) # make this a NaN so the column can benumeric


,Date,Tm,Home_Away,Opp,W/L,R,RA,Inn,W-L,Rank,GB,Win,Loss,Save,Time,D/N,Attendance,cLI,Streak,Orig. Scheduled
1,"Sunday, Apr 2",ARI,Home,SFG,W-wo,6.0,5.0,9.0,1-0,1.0,up 0.5,Rodney,Melancon,None,3:23,D,49016.0,1.06,1,None
2,"Tuesday, Apr 4",ARI,Home,SFG,L,4.0,8.0,9.0,1-1,2.0,1.0,Cueto,Corbin,None,3:26,N,19378.0,1.12,-1,None
3,"Wednesday, Apr 5",ARI,Home,SFG,W,8.0,6.0,9.0,2-1,1.0,Tied,Walker,Moore,Rodney,3:15,N,14675.0,1.03,1,None
4,"Thursday, Apr 6",ARI,Home,SFG,W,9.0,3.0,9.0,3-1,1.0,Tied,De La Rosa,Samardzija,None,3:00,N,15308.0,1.10,2,None
5,"Friday, Apr 7",ARI,Home,CLE,W,7.0,3.0,9.0,4-1,1.0,Tied,Miller,Tomlin,None,3:19,N,22443.0,1.00,3,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158,"Tuesday, Sep 26",ARI,Home,SFG,W,11.0,4.0,9.0,91-67,2.0,10.0,Ray,Moore,None,3:23,N,27487.0,.02,1,None
159,"Wednesday, Sep 27",ARI,Home,SFG,W-wo,4.0,3.0,9.0,92-67,2.0,10.0,Hoover,Dyson,None,3:20,D,20048.0,.03,2,None
160,"Friday, Sep 29",ARI,@,KCR,L,1.0,2.0,9.0,92-68,2.0,10.0,Kennedy,Greinke,Minor,2:59,N,23488.0,.04,-1,None
161,"Saturday, Sep 30",ARI,@,KCR,L,3.0,4.0,9.0,92-69,2.0,11.0,Junis,Bradley,Minor,2:44,N,32727.0,.05,-2,None


In [7]:
print(train_data)

{'team1': ['ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'ARI', 'A